In [1]:
import simpy
import random 
import numpy as np

In [2]:
class JusticeSystem():
    def __init__(self, env) -> None:
        self.active_cases = 0
        self.magistrate_cases = 0
        self.env = env

        # Some random, hard-coded constants
        self.summoned_dismissed_prob = 0.3
        self.magistrate_dismissed_prob = 0.3
        self.magistrate_sentenced_prob = 0.5 # This is after not dismissing. 0.5 of the `legitimate` cases got sentenced


    # The summoned process
    def summoned(self, env):
        while True:
            # Influx of cases
            self.active_cases += 1

            # State at the beginning of the day
            print(f'At the beginning of day {env.now+1}, the number of active cases is {self.active_cases},000 cases')

            # Cases dismissed
            if np.random.uniform() < self.summoned_dismissed_prob:
                self.active_cases -= 1
                yield self.env.timeout(1)
            
            # Case moved to magistrate
            else:
                yield self.env.timeout(1) # This means it takes 1 day to move from summoned to magistrate
                yield self.env.process(self.magistrate()) 
            

    # The magistrate process
    def magistrate(self):
        # Incoming magistrate cases
        self.magistrate_cases += 1

        # Cases dismissed
        if np.random.uniform() < self.magistrate_dismissed_prob:
            if self.magistrate_cases > 0: 
                self.magistrate_cases -= 1
                self.active_cases -= 1
                yield self.env.timeout(1)    
            else: yield self.env.timeout(1)

        else:
            # Cases sentenced
            yield self.env.timeout(1)
            if np.random.uniform() < self.magistrate_sentenced_prob:

In [3]:
def simulate(env):
    justice_system = JusticeSystem(env)
    env.process(justice_system.summoned(env)) #Add process `summoned` to the simpy env
    
    yield env.timeout(0)

In [4]:
env = simpy.Environment()

env.process(simulate(env)) # Add process `simulate` to the simpy env
env.run(until=100)

At the beginning of day 1, the number of active cases is 1,000 cases
At the beginning of day 2, the number of active cases is 1,000 cases
At the beginning of day 4, the number of active cases is 2,000 cases
At the beginning of day 5, the number of active cases is 2,000 cases
At the beginning of day 6, the number of active cases is 2,000 cases
At the beginning of day 7, the number of active cases is 2,000 cases
At the beginning of day 9, the number of active cases is 3,000 cases
At the beginning of day 10, the number of active cases is 3,000 cases
At the beginning of day 11, the number of active cases is 3,000 cases
At the beginning of day 13, the number of active cases is 4,000 cases
At the beginning of day 15, the number of active cases is 5,000 cases
At the beginning of day 17, the number of active cases is 6,000 cases
At the beginning of day 18, the number of active cases is 6,000 cases
At the beginning of day 20, the number of active cases is 7,000 cases
At the beginning of day 22,